[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/money-market-fund-nmfp-python.ipynb)

# Analyze Money Market Fund Holdings from SEC N-MFP Filings with Python -- Free, No API Key

Use **edgartools** to analyze money market fund portfolio holdings from SEC N-MFP filings -- completely free, no API key or paid subscription required. Money market funds file monthly N-MFP reports disclosing every security they hold, plus yield, NAV, and liquidity data.

**What you'll learn:**
- Parse an N-MFP filing to see every holding in a money market fund
- Get fund metadata: net assets, WAM, WAL, fund category
- Analyze portfolio holdings by investment category
- Track yield, NAV, and liquidity time series
- Explore share class data and repo collateral
- Compare money market funds side by side

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Parse a Money Market Fund Filing

JPMorgan's Prime Money Market Fund is one of the largest in the US. Parse its latest N-MFP3 filing to see the full portfolio:

In [ ]:
# Find the latest N-MFP3 filing for JPMorgan Prime Money Market Fund
filings = get_filings(form="N-MFP3").filter(company_name="JPMORGAN TRUST I")
mmf = filings[0].obj()

print(f"Fund:          {mmf.name}")
print(f"Report Date:   {mmf.report_date}")
print(f"Category:      {mmf.fund_category}")
print(f"Net Assets:    ${float(mmf.net_assets):,.0f}")
print(f"WAM (days):    {mmf.average_maturity_wam}")
print(f"WAL (days):    {mmf.average_maturity_wal}")
print(f"Securities:    {mmf.num_securities}")
print(f"Share Classes: {mmf.num_share_classes}")

## Rich Display

The `MoneyMarketFund` object has a built-in Rich display showing the fund summary, share classes, and top holdings:

In [ ]:
mmf

## Portfolio Holdings

Get all portfolio securities as a DataFrame. Each row is a security with issuer, CUSIP, investment category, yield, market value, and more:

In [ ]:
portfolio = mmf.portfolio_data()

# Top 15 holdings by market value
top = portfolio.head(15)[["issuer", "cusip", "category", "yield", "market_value", "pct_of_nav"]].copy()
top["market_value"] = top["market_value"].apply(lambda v: f"${float(v):,.0f}" if pd.notna(v) else "")
top["pct_of_nav"] = top["pct_of_nav"].apply(lambda v: f"{float(v):.2%}" if pd.notna(v) else "")
top

## Holdings by Investment Category

See how the fund allocates across asset types -- Treasury, Agency, Repo, Commercial Paper, Certificates of Deposit, etc.:

In [ ]:
categories = mmf.holdings_by_category()
categories["total_market_value"] = categories["total_market_value"].apply(
    lambda v: f"${float(v):,.0f}" if pd.notna(v) else ""
)
categories["total_pct"] = categories["total_pct"].apply(
    lambda v: f"{float(v):.2%}" if pd.notna(v) else ""
)
categories.rename(columns={
    "category": "Category",
    "count": "Holdings",
    "total_market_value": "Market Value",
    "total_pct": "% of NAV",
})

## Yield History

N-MFP3 filings include a daily time series of 7-day gross yields (up to 20 business days). Track how the fund's yield has changed over the reporting period:

In [ ]:
yields = mmf.yield_history()
if not yields.empty:
    print(f"Yield data points: {len(yields)}")
    print(f"Date range: {yields['date'].min()} to {yields['date'].max()}")
    yields.head(10)

## NAV History

Track daily net asset value per share. For stable-NAV funds this stays near $1.0000, but floating-NAV funds may show small fluctuations:

In [ ]:
nav = mmf.nav_history()
if not nav.empty:
    print(f"NAV data points: {len(nav)}")
    nav.head(10)

## Liquidity Analysis

Money market funds must maintain daily and weekly liquid asset minimums. Track these over the reporting period:

In [ ]:
liquidity = mmf.liquidity_history()
if not liquidity.empty:
    print(f"Liquidity data points: {len(liquidity)}")
    liquidity.head(10)

## Share Classes

Large money market funds often have multiple share classes with different minimum investments and fee structures:

In [ ]:
shares = mmf.share_class_data()
shares["net_assets"] = shares["net_assets"].apply(lambda v: f"${float(v):,.0f}" if pd.notna(v) else "")
shares["min_investment"] = shares["min_investment"].apply(lambda v: f"${float(v):,.0f}" if pd.notna(v) else "")
shares.rename(columns={
    "class_name": "Class",
    "class_id": "ID",
    "min_investment": "Min Investment",
    "net_assets": "Net Assets",
    "shares_outstanding": "Shares Outstanding",
})

## Repo Collateral

Many money market funds hold repurchase agreements. The collateral backing each repo is disclosed in the filing:

In [ ]:
collateral = mmf.collateral_data()
if not collateral.empty:
    print(f"Repo collateral items: {len(collateral)}")
    collateral.head(10)
else:
    print("No repurchase agreements in this filing")

## Compare Money Market Funds

Pull N-MFP data from multiple funds to compare size, maturity, and category:

In [ ]:
# Search for several large money market fund filings
fund_names = ["JPMORGAN TRUST I", "FIDELITY", "VANGUARD"]
rows = []

for name in fund_names:
    filing = get_filings(form="N-MFP3").filter(company_name=name)
    if len(filing) > 0:
        fund = filing[0].obj()
        rows.append({
            "Fund": fund.name[:50],
            "Category": fund.fund_category or "N/A",
            "Net Assets ($B)": f"{float(fund.net_assets) / 1e9:.1f}" if fund.net_assets else "N/A",
            "WAM (days)": fund.average_maturity_wam,
            "WAL (days)": fund.average_maturity_wal,
            "Securities": fund.num_securities,
        })

pd.DataFrame(rows)

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing N-MFP money market fund data:

**With edgartools (free, no API key):**
```python
mmf = filing.obj()
mmf.portfolio_data()        # All holdings as DataFrame
mmf.yield_history()         # Daily 7-day gross yields
mmf.holdings_by_category()  # Asset allocation breakdown
```

**Typical approach (manual XML parsing):**
```python
from lxml import etree
# ... download N-MFP XML from EDGAR,
# ... handle different schemas for N-MFP2 vs N-MFP3,
# ... strip XML namespaces, extract nested time series,
# ... parse repo collateral hierarchies manually
```

With edgartools, N-MFP data is parsed automatically -- portfolio holdings, yields, NAV, liquidity, and collateral accessible as pandas DataFrames.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Parse an N-MFP filing ──
filing = get_filings(form="N-MFP3").filter(company_name="JPMORGAN")[0]
mmf = filing.obj()

# ── Fund metadata ──
mmf.name                     # Fund series name
mmf.report_date              # Report date
mmf.fund_category            # Prime, Government, Tax-Exempt
mmf.net_assets               # Total net assets
mmf.average_maturity_wam     # Weighted avg maturity (days)
mmf.average_maturity_wal     # Weighted avg life (days)
mmf.num_securities           # Number of portfolio holdings
mmf.num_share_classes        # Number of share classes

# ── DataFrames ──
mmf.portfolio_data()         # All holdings with yield, value, % NAV
mmf.holdings_by_category()   # Holdings grouped by investment type
mmf.share_class_data()       # Share class summary
mmf.collateral_data()        # Repo collateral details
mmf.yield_history()          # 7-day gross yield time series
mmf.nav_history()            # Daily NAV per share
mmf.liquidity_history()      # Daily/weekly liquid asset %
```

## What's Next

You've learned how to analyze money market fund holdings from SEC N-MFP filings. Here are related tutorials:

- [Mutual Fund Holdings (N-PORT)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/mutual-fund-holdings-nport-python.ipynb)
- [ETF and Fund Holdings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/etf-fund-holdings-python.ipynb)
- [13F Institutional Holdings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/13f-institutional-holdings-python.ipynb)
- [SEC EDGAR API in Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-edgar-api-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with money market fund data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*